## R6 Inheritance

Learn how to inherit from an R6 class, and how the relationship between parent and child classes works.

### Specifying a Fancy Microwave Oven
Inheritance is used to propagate – that is, copy – functionality from one class to another. To create a child class from another class, use the inherit argument to R6Class().

The pattern to create a child class is:

child_class_factory <- R6Class(
  
  "ChildClass",
  
  inherit = parent_class_factory
)

In [4]:
library(R6)
library(assertive)

microwave_oven_factory <- R6Class(
  "MicrowaveOven",
  private = list(
    ..power_rating_watts = 800,
    ..power_level_watts = 800
  ),
  
  public = list(
    cook = function(time_seconds) {
      Sys.sleep(time_seconds)
      print("Your food is cooked!")
    }),
  # Add active list containing an active binding
  active = list(
    power_level_watts = function(value) {
      if(missing(value)) {
        # Return the private value
        private$..power_level_watts
      } else {
        # Assert that value is a number
        assert_is_a_number(value)
        # Assert that value is in a closed range from 0 to power rating
        assert_all_are_in_closed_range(value, 0,private$..power_rating_watts)
        # Set the private power level to value
        private$..power_level_watts <- value
      }
    }
  )
)



# Define a fancy microwave class inheriting from microwave oven
fancy_microwave_oven_factory <- R6Class(
    "FancyMicrowaveOven",
    inherit = microwave_oven_factory
)

### Making a Fancy Microwave Oven
Inheritance means that the methods of the child class are exact copies of those in the parent class.

In [5]:
# Explore microwave oven classes
microwave_oven_factory
fancy_microwave_oven_factory

# Instantiate both types of microwave
a_microwave_oven <- microwave_oven_factory$new()
a_fancy_microwave <- fancy_microwave_oven_factory$new()

# Get power rating for each microwave
microwave_power_rating <- a_microwave_oven$power_rating_watts
fancy_microwave_power_rating <- a_fancy_microwave$power_rating_watts

# Verify that these are the same
identical(microwave_power_rating, fancy_microwave_power_rating)

# Cook with each microwave
a_microwave_oven$cook(1)
a_fancy_microwave$cook(1)

<MicrowaveOven> object generator
  Public:
    cook: function (time_seconds) 
    clone: function (deep = FALSE) 
  Active bindings:
    power_level_watts: function (value) 
  Private:
    ..power_rating_watts: 800
    ..power_level_watts: 800
  Parent env: <environment: R_GlobalEnv>
  Locked objects: TRUE
  Locked class: FALSE
  Portable: TRUE

<FancyMicrowaveOven> object generator
  Inherits from: <microwave_oven_factory>
  Public:
    clone: function (deep = FALSE) 
  Parent env: <environment: R_GlobalEnv>
  Locked objects: TRUE
  Locked class: FALSE
  Portable: TRUE

[1] TRUE

[1] "Your food is cooked!"
[1] "Your food is cooked!"


### Extending the Cooking Capabilities
The child class can extend the functionality of the parent by adding further public methods with names that are different to those available in the parent class.

Public methods can call other public methods by prefixing their name with self$.

In [6]:
# Extend the class definition
fancy_microwave_oven_factory <- R6Class(
  "FancyMicrowaveOven",
  inherit = microwave_oven_factory,
  # Add a public list with a cook baked potato method
  public = list(
    cook_baked_potato = function(){
      self$cook(3)
    }
  )
)

# Instantiate a fancy microwave
a_fancy_microwave <- fancy_microwave_oven_factory$new()

# Call the cook_baked_potato() method
a_fancy_microwave$cook_baked_potato()

[1] "Your food is cooked!"


### Overriding the Cooking Capabilities
Child classes can also extend functionality by overriding methods. They do this by defining methods with the same name as that of the parent.

Child classes can access public methods from their parent class by prefixing the name with super$.

In [7]:
# Update the class definition
fancy_microwave_oven_factory <- R6Class(
  "FancyMicrowaveOven",
  inherit = microwave_oven_factory,
  # Add a public list with a cook method
  public = list(
    cook = function(time_seconds){
      super$cook(time_seconds)
      message("Enjoy your dinner!")
    } 
  )
)

# Instantiate a fancy microwave
a_fancy_microwave <- fancy_microwave_oven_factory$new()

# Call the cook() method
a_fancy_microwave$cook(1)

[1] "Your food is cooked!"


Enjoy your dinner!


### Exposing your Parent
By default, R6 classes only have access to the functionality of their direct parent. To allow access across multiple generations, the intermediate classes need to define an active binding that exposes their parent. This takes the form

active = list(
  
  super_ = function() super

)

In [8]:
# Expose the parent functionality
fancy_microwave_oven_factory <- R6Class(
  "FancyMicrowaveOven",
  inherit = microwave_oven_factory,
  public = list(
    cook_baked_potato = function() {
      self$cook(3)
    },
    cook = function(time_seconds) {
      super$cook(time_seconds)
      message("Enjoy your dinner!")
    }
  ),
  # Add an active element with a super_ binding
  active = list(
    super_ = function() super
  )
)

# Instantiate a fancy microwave
a_fancy_microwave <- fancy_microwave_oven_factory$new()

# Call the super_ binding
a_fancy_microwave$super_$cook(1)

[1] "Your food is cooked!"


### Over-Overriding the Cooking Capabilities

Once intermediate classes have exposed their parent functionality with super_ active bindings, you can access methods across several generations of R6 class. The syntax is

parent_method <- super$method()

grand_parent_method <- super$super_

$method()

great_grand_parent_method <- super$super_

$super_

$method()

In [10]:
# Explore other microwaves
microwave_oven_factory
fancy_microwave_oven_factory

# Define a high-end microwave oven class
high_end_microwave_oven_factory <- R6Class(
 "HighEndMicrowaveOven",
  inherit = fancy_microwave_oven_factory,
  public = list(
    cook = function(time_seconds){
      super$super_$cook(time_seconds)
       message("enjoy your food")
    }
  )
)

# Instantiate a high-end microwave oven
a_high_end_microwave <- high_end_microwave_oven_factory$new()

# Use it to cook for one second
a_high_end_microwave$cook(1)

<MicrowaveOven> object generator
  Public:
    cook: function (time_seconds) 
    clone: function (deep = FALSE) 
  Active bindings:
    power_level_watts: function (value) 
  Private:
    ..power_rating_watts: 800
    ..power_level_watts: 800
  Parent env: <environment: R_GlobalEnv>
  Locked objects: TRUE
  Locked class: FALSE
  Portable: TRUE

<FancyMicrowaveOven> object generator
  Inherits from: <microwave_oven_factory>
  Public:
    cook_baked_potato: function () 
    cook: function (time_seconds) 
    clone: function (deep = FALSE) 
  Active bindings:
    super_: function () 
  Parent env: <environment: R_GlobalEnv>
  Locked objects: TRUE
  Locked class: FALSE
  Portable: TRUE

[1] "Your food is cooked!"


enjoy your food
